# Use Close-Form Policy to Play Pendulum-v0

In [1]:
%matplotlib inline

import sys
import logging
import imp
import itertools

import numpy as np
np.random.seed(0)
import pandas as pd
import gym

imp.reload(logging)
logging.basicConfig(level=logging.DEBUG,
        format='%(asctime)s [%(levelname)s] %(message)s',
        stream=sys.stdout, datefmt='%H:%M:%S')

In [2]:
env = gym.make('Pendulum-v0')
env.seed(0)
for key in vars(env):
    logging.info('%s: %s', key, vars(env)[key])

09:17:16 [INFO] env: <PendulumEnv<Pendulum-v0>>
09:17:16 [INFO] action_space: Box(1,)
09:17:16 [INFO] observation_space: Box(3,)
09:17:16 [INFO] reward_range: (-inf, inf)
09:17:16 [INFO] metadata: {'render.modes': ['human', 'rgb_array'], 'video.frames_per_second': 30}
09:17:16 [INFO] _max_episode_steps: 200
09:17:16 [INFO] _elapsed_steps: None


In [3]:
class CloseFormAgent:
    def __init__(self, _):
        pass

    def reset(self, mode=None):
        pass

    def step(self, observation, _reward, _done):
        x, y, angle_velocity = observation
        flip = (y < 0.)
        if flip:
            y *= -1. # now y >= 0
            angle_velocity *= -1.
        angle = np.arcsin(y)
        if x < 0.:
            angle = np.pi - angle
        if (angle < -0.3 * angle_velocity) or \
                (angle > 0.03 * (angle_velocity - 2.5) ** 2. + 1. and \
                angle < 0.15 * (angle_velocity + 3.) ** 2. + 2.):
            force = 2.
        else:
            force = -2.
        if flip:
            force *= -1.
        action = np.array([force,])
        return action

    def close(self):
        pass


agent = CloseFormAgent(env)

In [4]:
def play_episode(env, agent, max_episode_steps=None, mode=None, render=False):
    observation, reward, done = env.reset(), 0., False
    agent.reset(mode=mode)
    episode_reward, elapsed_steps = 0., 0
    while True:
        action = agent.step(observation, reward, done)
        if render:
            env.render()
        if done:
            break
        observation, reward, done, _ = env.step(action)
        episode_reward += reward
        elapsed_steps += 1
        if max_episode_steps and elapsed_steps >= max_episode_steps:
            break
    agent.close()
    return episode_reward, elapsed_steps


logging.info('==== test ====')
episode_rewards = []
for episode in range(100):
    episode_reward, elapsed_steps = play_episode(env, agent)
    episode_rewards.append(episode_reward)
    logging.debug('test episode %d: reward = %.2f, steps = %d',
            episode, episode_reward, elapsed_steps)
logging.info('average episode reward = %.2f ± %.2f',
        np.mean(episode_rewards), np.std(episode_rewards))

09:17:16 [INFO] ==== test ====
09:17:16 [DEBUG] test episode 0: reward = -267.55, steps = 200
09:17:16 [DEBUG] test episode 1: reward = -124.84, steps = 200
09:17:16 [DEBUG] test episode 2: reward = -235.76, steps = 200
09:17:16 [DEBUG] test episode 3: reward = -126.20, steps = 200
09:17:16 [DEBUG] test episode 4: reward = -231.72, steps = 200
09:17:16 [DEBUG] test episode 5: reward = -283.73, steps = 200
09:17:16 [DEBUG] test episode 6: reward = -2.03, steps = 200
09:17:16 [DEBUG] test episode 7: reward = -225.50, steps = 200
09:17:16 [DEBUG] test episode 8: reward = -2.29, steps = 200
09:17:16 [DEBUG] test episode 9: reward = -283.62, steps = 200
09:17:16 [DEBUG] test episode 10: reward = -120.88, steps = 200
09:17:16 [DEBUG] test episode 11: reward = -3.70, steps = 200
09:17:16 [DEBUG] test episode 12: reward = -237.45, steps = 200
09:17:16 [DEBUG] test episode 13: reward = -119.87, steps = 200
09:17:16 [DEBUG] test episode 14: reward = -1.69, steps = 200
09:17:17 [DEBUG] test episo

In [5]:
env.close()